## Getting json files and loading their body text for filtering about target topic

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

import pickle
from gensim import corpora, models
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
from nltk.corpus import stopwords
import pickle
import en_core_web_sm
import csv
import json

from sklearn.preprocessing import Binarizer
from gensim.corpora import Dictionary
from gensim.models import TfidfModel


In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from scipy.spatial.distance import cosine, cdist

In [ ]:
path = '../input/CORD-19-research-challenge/'
meta_df = pd.read_csv(path + 'metadata.csv') 
meta_df.head()

In [ ]:
print("Cols names: {}".format(meta_df.columns))

In [ ]:
len(meta_df)

In [ ]:
meta_for_merge = meta_df[['title', 'abstract','pdf_json_files','url']]
#meta_abstracts = meta_abstracts.rename({'title':'Study', 'abstract':'Abstract'}, axis=1)
meta_for_merge.head()

In [ ]:
smoke_df = pd.read_csv(path +'Kaggle/target_tables/8_risk_factors/' + 'Smoking Status.csv', index_col= 'Unnamed: 0') 
smoke_df.head()

In [ ]:
print("Cols names: {}".format(smoke_df.columns))

In [ ]:
len(smoke_df)

In [ ]:
smoke_to_merge = smoke_df[['Date', 'Study', 'Journal']]

In [ ]:
smoke_target = smoke_to_merge.merge(meta_for_merge,how='inner', left_on='Study', right_on='title')
smoke_target.head()

In [ ]:
len(smoke_target)

In [ ]:
plt.figure(figsize=(20,10))
smoke_target.isna().sum().plot(kind='bar', stacked=True)

In [ ]:
smoke_target.drop(columns=['title'], inplace=True)
smoke_target.drop_duplicates(subset='Study', keep="first", inplace=True)
len(smoke_target)

In [ ]:
smoke_target.dropna(axis=0, inplace=True)
smoke_target.reset_index(drop=True, inplace=True)
smoke_target.head()

In [ ]:
len(smoke_target)

### Beginning loading json files

In [ ]:
# example with the second file from the target table
with open(path + smoke_target.pdf_json_files[1], 'r') as myfile:
    data=myfile.read()

# parse file
obj = json.loads(data)
# convert json body_text into a text list
body = obj['body_text']
just_text2 = [body[d]['text'] for d in range(len(body))]
print(len(just_text2))

In [ ]:
# Transform all data to lower-case
clean_body = [text.lower() for text in just_text2]
clean_body = [strip_numeric(text) for text in clean_body] # Remove numbers
clean_body = [strip_punctuation(text) for text in clean_body] # Remove punctuation
clean_body = [strip_multiple_whitespaces(text) for text in clean_body] # Remove multiple spaces
clean_body = [remove_stopwords(text) for text in clean_body] #removing the stopwords
clean_body = [strip_short(text) for text in clean_body] #removing words with length lesser than 3 
print(clean_body[0])

### Filtering for smoking topic 

In [ ]:
# testing the parsing loop for smoking
att = ['he smokes a lot', 'i dont like smoking', 'who is smoking', 'what is to smoke', 'a test here it is smok']
for text in att:
    if 'smok' in text:
        print(text)

In [ ]:
# check if some part of the body is related to smoking and print the index and the part
for i in range(len(clean_body)):
    if 'smok' in clean_body[i]:
        print(i)
        print(clean_body[i])

## Final function for getting the relevant documents from json files

In [ ]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

In [ ]:
smoke_target['original']=''
smoke_target['body']=''
smoke_target['sent']=''
smoke_target['sent2']=''
  
def countOccurences(str, word): 
      
    # split the string by spaces in a 
    a = str.split(" ") 
  
    # search for pattern in a 
    count = 0
    for i in range(0, len(a)): 
          
        # if match found increase count  
        if (word == a[i]): 
           count = count + 1
             
    return count        
y=''  

In [ ]:
def summarize_docs(dataframe):
    for i in range(len(smoke_target)):
        try:
            # open json file
            with open(path + smoke_target.pdf_json_files[i], 'r') as myfile:
                data=myfile.read()
            # parse file
            obj = json.loads(data)
        except:
            ValueError
        body = obj['body_text']
        body = [body[d]['text'] for d in range(len(body))]
        y=sent_tokenize(str(body))
        df=pd.DataFrame(index=np.arange(len(y)))
        df['content']='none'
        df['original']=df['content']
      
        for j in range(len(y)):
            df.original[j]=y[j] 
            df.content[j]=y[j]
          
        for i in range(len(df.content)):
            df.content[i] =  strip_numeric(df.content[i]) #Remove digits
            df.content[i] =  strip_punctuation(df.content[i])  #Remove punctuation
            df.content[i] =  strip_multiple_whitespaces(df.content[i]) #Remove multiple whitespaces
            df.content[i] =  df.content[i].lower()
            df.content[i] =  remove_stopwords(df.content[i])
            df.content[i] =  strip_short(df.content[i])
            df.content[i] =  stem_text(df.content[i])
        df['index']=df.index
        df['count']= 0
        for i in range(len(df.content)):
            str = df.content[i]
            word ="smoke"
            df['count'][i]=float((countOccurences(str, word))*1000/len(df.content[i]))
        x=df.loc[df['count'].idxmax()]
        summary=df.original[x['index']]
        return df

In [ ]:
final_smoke_relevant_docs = summarize_docs(smoke_target)
#for i in range(len(final_smoke_relevant_docs)):
 #   print(final_smoke_relevant_docs[i])
summarize_docs(smoke_target)

In [ ]:
smoke_target.tail()

In [ ]:
def summarize_docs(dataframe, target):
    """
    This function will get the relevant docs from a dataframe depending on a target subject.
    
    The dataframe needs to have a column 'pdf_json_files' on it, containing json files names.
    The target is in str format.
    
    
    At the end it will print the number of relevant docs and 
    return a list with:
        [1] = index from table, 
        [2] = indexes from the body list, 
        [3] = the preprocessed body text list
    
    """
    # parsing through all the docs in the target table
    related_docs = []
    for i in range(len(dataframe)):
        try:
            # open json file
            with open(path + dataframe.pdf_json_files[i], 'r') as myfile:
                data=myfile.read()
            # parse file
            obj = json.loads(data)
        except:
            ValueError
        # convert json body_text into a text list
        smoke_target['body_dirty']=''
        body = obj['body_text']
        just_text = sent_tokenize([body[d]['text'] for d in range(len(body))])
        smoke_target['body_dirty']= sent_tokenize([body[d]['text'])
        clean_body = [text.lower() for text in just_text]
        clean_body = [strip_numeric(text) for text in clean_body] # Remove numbers
        clean_body = [strip_punctuation(text) for text in clean_body] # Remove punctuation
        clean_body = [strip_multiple_whitespaces(text) for text in clean_body] # Remove multiple spaces
        clean_body = [remove_stopwords(text) for text in clean_body] #removing the stopwords
        clean_body = [strip_short(text) for text in clean_body]
        y=sent_tokenize(body)
        relevant_parts = []
        # check if the doc is related with smoking
        for t in range(len(clean_body)):
            if target in clean_body[t]:
                # save the index of relevant parts
                relevant_parts.append(t)
                # save the docs in a list that has: target_index, clean_body_index, clean_json_body 
        if len(relevant_parts) != 0:
            related_docs.append([i, relevant_parts, clean_body])

    print('You have ' + str(len(related_docs)) + ' relevant docs')
    return related_docs

In [ ]:
def get_relevant_docs(dataframe, target):
    """
    This function will get the relevant docs from a dataframe depending on a target subject.
    
    The dataframe needs to have a column 'pdf_json_files' on it, containing json files names.
    The target is in str format.
    
    
    At the end it will print the number of relevant docs and 
    return a list with:
        [1] = index from table, 
        [2] = indexes from the body list, 
        [3] = the preprocessed body text list
    
    """
    # parsing through all the docs in the target table
    related_docs = []
    for i in range(len(dataframe)):
        try:
            # open json file
            with open(path + dataframe.pdf_json_files[i], 'r') as myfile:
                data=myfile.read()
            # parse file
            obj = json.loads(data)
        except:
            ValueError
        # convert json body_text into a text list
        body = obj['body_text']
        just_text = [body[d]['text'] for d in range(len(body))]
        clean_body = [text.lower() for text in just_text]
        clean_body = [strip_numeric(text) for text in clean_body] # Remove numbers
        clean_body = [strip_punctuation(text) for text in clean_body] # Remove punctuation
        clean_body = [strip_multiple_whitespaces(text) for text in clean_body] # Remove multiple spaces
        clean_body = [remove_stopwords(text) for text in clean_body] #removing the stopwords
        clean_body = [strip_short(text) for text in clean_body]

        relevant_parts = []
        # check if the doc is related with smoking
        for t in range(len(clean_body)):
            if target in clean_body[t]:
                # save the index of relevant parts
                relevant_parts.append(t)
                # save the docs in a list that has: target_index, clean_body_index, clean_json_body 
        if len(relevant_parts) != 0:
            related_docs.append([i, relevant_parts, clean_body])

    print('You have ' + str(len(related_docs)) + ' relevant docs')
    return related_docs

In [ ]:
final_smoke_relevant_docs = get_relevant_docs(smoke_target, 'smok')
print(final_smoke_relevant_docs[1])

In [ ]:
# parsing through all the docs in the target table

smoke_related_docs = [['target_index', 'relevant_index', 'clean_json_body']] # the first object is the description of the list content
for i in range(len(smoke_target)):
    try:
        # open json file
        with open(path + smoke_target.pdf_json_files[i], 'r') as myfile:
            data=myfile.read()
        # parse file
        obj = json.loads(data)
    except:
        ValueError
    # convert json body_text into a text list
    body = obj['body_text']
    just_text = [body[d]['text'] for d in range(len(body))]
    clean_body = [text.lower() for text in just_text]
    
    relevant_parts = []
    # check if the doc is related with smoking
    for t in range(len(clean_body)):
        if 'smok' in clean_body[t]:
            # save the index of relevant parts
            relevant_parts.append(t)
            # save the docs in a list that has: target_index, clean_body_index, clean_json_body 
    if len(relevant_parts) != 0:
        smoke_related_docs.append([i, relevant_parts, clean_body])

print(len(smoke_related_docs))

In [ ]:
print(smoke_related_docs[3])

In [ ]:
# it seems like the smoke_target.pdf_json_files[2] has two json files related to it
for i in range(len(smoke_target)):
    print(smoke_target.pdf_json_files[i])

In [ ]:
# for examination purpose we take just one
smoke_target.pdf_json_files[2] = 'document_parses/pdf_json/7a4a40db618de7c1be7eda37e30ad1eb03c16c21.json'

In [ ]:
# Stemming
smoke_stem=[stem_text(abstract) for abstract in smoke_data]
print(smoke_stem[4])

In [ ]:
#stemmed corpus storage
pickle.dump(smoke_stem, open("smoke_stemmed.pkl", "wb"))